In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import os, glob
cwd = os.getcwd()
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
im = cv2.imread('/content/drive/MyDrive/test/home.jpg')
im=cv2.resize(im, (1434,1080))

In [ ]:
arr=[]
for i in range(10): 
    cv2.rectangle(im,(120+80*i,872),(200+80*i,1020),(0,0,255))
    arr.append(((120+80*i,872,200+80*i,1020))) #x1 y1, x2, y2순으로 arr에 저장
for i in range(14):
    cv2.rectangle(im,(300+50*i,299-i//3),(350+50*i,360-i//3),(0,0,255))
    arr.append(((300+50*i,299-i//3,350+50*i,360-i//3)))

In [ ]:
cv2_imshow(im)

In [ ]:
arr

In [ ]:
from PIL import Image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
cwd = os.getcwd()
top_model_weights_path = '/content/drive/MyDrive/test/model.h5'

class_dictionary = {}
class_dictionary[0] = 'full'
class_dictionary[1] = 'empty'
class_dictionary[2] = 'Disabled'

In [ ]:
model = load_model(top_model_weights_path)

In [ ]:
def make_prediction(image):
    img = image/255.
    #convert to a 4D tensor
    image = np.expand_dims(img, axis=0)
    #print(image.shape)

    # make prediction
    class_predicted = model.predict(image)
    inID = np.argmax(class_predicted[0])
    label = class_dictionary[inID]
    return label

In [ ]:
def predict_on_image(im, spot_dict = arr, make_copy=True, color = [0, 255, 0], alpha=0.5):
    if make_copy:
        new_image = np.copy(im)
        overlay = np.copy(im)
    cnt_empty = 0
    cnt_disabled=0
    all_spots = 0

    for spot in spot_dict:
        all_spots += 1
        (x1, y1, x2, y2) = spot
        (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
        #crop this image
        spot_img = im[y1:y2, x1:x2]
        spot_img = cv2.resize(spot_img, (224, 224))

        label = make_prediction(spot_img)
        if label == 'empty':
            cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), color,-1)
            cnt_empty += 1
        elif label == 'full':
            cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), (0,0,255),-1)
        else:
            cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),-1)
            cnt_disabled+=1
            
            
    cv2.addWeighted(overlay, alpha, new_image, 1 - alpha, 0, new_image)
    
    cv2.putText(new_image, "Total: %d spots" %all_spots, (30, 55),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (0, 0, 0), 2)
    save = False

    cv2.putText(new_image, "Available: %d spots" %cnt_empty, (30, 85),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (255, 255, 255), 2)

    cv2.putText(new_image, "Disabled: %d spots" %cnt_disabled, (30, 115),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (255, 255, 255), 2)
    
    if save:
        filename = 'with_marking.jpg'
        cv2.imwrite(filename, new_image)
    
    return new_image


predicted_images = predict_on_image(im, arr, make_copy=True, color = [0, 255, 0], alpha=0.5)
predicted_images=cv2.resize(predicted_images, (700,500))
cv2_imshow(predicted_images)

In [ ]:
   spot_img = im[y1:y2, x1:x2]
    spot_img = cv2.resize(spot_img, (224, 224))
    (200, 872, 280, 1020)

In [ ]:
new=im[872:1020,1200:1300]
cv2_imshow(new)
new=cv2.resize(new,(224,224))
img = new/255.
new = np.expand_dims(img, axis=0)
a=model.predict(new)
a